In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time
import cv2
from sklearn import metrics
import gc

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models


In [ ]:
folder_train = "/kaggle/input/histopathologic-cancer-detection/train/"
folder_test = "/kaggle/input/histopathologic-cancer-detection/test/"

sample_df = pd.read_csv("/kaggle/input/histopathologic-cancer-detection/sample_submission.csv")
label_df = pd.read_csv("/kaggle/input/histopathologic-cancer-detection/train_labels.csv")

train_name = label_df["id"].to_numpy()
test_name =  sample_df["id"].to_numpy()
train_label = label_df["label"].to_numpy()

def plot_photo(N, nc = 4, pos = True):
    
    nr =  math.ceil(N/nc)
    n_train = train_name.shape[0]
    fig, ax = plt.subplots(nr, nc, figsize = (15, nr*3.2))
    
    np.random.seed(1)
    
    IDs = np.random.choice(n_train, N, replace = False)
    
    if pos:
        filter1 = train_label == 1
    else:
        filter1 = train_label == 0
        
    names_1 = train_name[filter1]
    
    names = np.random.choice(names_1, N, replace = False)
    
    for k in range(N):
        i =  int(k/nc)
        j = k % nc
        fname =  folder_train + names[k] + ".tif"
        img = cv2.imread(fname)
        ax[i,j].imshow(img)